In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from tqdm.notebook import tqdm
import time
import sys
import datetime
import time

In [19]:
driver = webdriver.Chrome(executable_path = r'C:\Users\Administrator\Downloads\chromedriver.exe')

In [3]:
df1 = pd.read_excel(r'Philips Top Products.xlsx',header = 0,engine = 'openpyxl')

In [4]:
# Need to get catalog images
# driver.get('https://www.flipkart.com/product/p/itme?pid=MOBGYFAXFPHUPEE5')
def getCatalogContent(driver):
    
    imgs = driver.find_elements_by_css_selector('div>ul>li>div>div[style*=background-image]')
    numVids = len(driver.find_elements_by_css_selector('div>ul>li>div>div:nth-child(3)'))
    imgList = []
    for i in imgs:
        url_ = i.value_of_css_property('background-image').strip('url("")').replace('128/','416/',2)
        imgList.append(url_)
    return imgList,numVids


In [ ]:
# Need to get all the remaining info on the webPage including features, description and Product title



In [5]:
# product_title = driver.find_element_by_css_selector('div>div>div>h1').text
def getTechnicalDetails(driver):
    details_fs = driver.find_elements_by_css_selector('table')
    df_prodTabularDetails = pd.DataFrame()
    productDetails = {}
    for tl in details_fs:
        table_content = tl.get_attribute('innerHTML')
        # pd.read_html(table_content)
        df = pd.read_html('<table>' + table_content + '</table>')[0]
        df_prodTabularDetails = df_prodTabularDetails.append(df,ignore_index=True)
    productDetails = df_prodTabularDetails.set_index(0)[1].to_dict()
    try:
        description = driver.find_elements_by_xpath('//div[contains(text(),"Description")]/following-sibling::div')
        driver.execute_script("arguments[0].setAttribute('style','max-height:None;overflow:None;')", description[0])
        productDetails['Description_product'] = description[0].text
    except:
        pass
    productDetails['timestamp'] = str(datetime.datetime.now())
    return productDetails



In [6]:
def getFeatureDetails(driver):
    feature_button = driver.find_elements_by_xpath("//button[text()='View all features']")
    if(len(feature_button)!=0):
        feature_button[0].click()
    details_elements = driver.find_elements_by_css_selector('div.col-12-12 div:nth-child(2)>p')

    featureDetails = {}
    for ftrIdx in range(len(details_elements)):
        featureDetails['feature_' + str(ftrIdx)] = details_elements[ftrIdx].text
    featureDetails['timestamp'] = str(datetime.datetime.now())
    return featureDetails


In [110]:
driver.get('https://www.flipkart.com/product/p/itme?pid=IRNFZV2YUBUZ3PTZ')

In [7]:
def getHlEmiDetails(driver):
    OtherDetails = driver.find_elements_by_css_selector('div.col-6-12')
    hl_emi_details = {}
    for hl_emi_idx in range(len(OtherDetails)):
        _detail = OtherDetails[hl_emi_idx].text
        detail_pair = _detail.split('\n',maxsplit = 1)
        if(_detail!='' and len(detail_pair)>1):
            hl_emi_details[detail_pair[0]] = detail_pair[1]
    hl_emi_details['timestamp'] = str(datetime.datetime.now())
    return hl_emi_details

In [8]:
def getASINData(asin,timeout,locator,driver):
    link = 'https://www.flipkart.com/product/p/itme?pid=' + str(asin)

    driver.get(link)
    WebDriverWait(driver, timeout).until(EC.visibility_of_element_located((By.CSS_SELECTOR, locator)))
    product_title = driver.find_element_by_css_selector('div>div>div>h1').text
    img_list,num_vids = getCatalogContent(driver)
    productDetails = getTechnicalDetails(driver)
    featureDetails = getFeatureDetails(driver)
    hl_emi_details =  getHlEmiDetails(driver)
    
    final_res = {}
    final_res['product title'] = product_title
    final_res['product details'] = productDetails
    final_res['feature details'] = featureDetails
    final_res['hl_emi details'] = hl_emi_details
    final_res['images'] = img_list
    final_res['num vids'] = num_vids
    final_res['timestamp'] = str(datetime.datetime.now())
    return final_res

In [9]:
def getData(asin_list,asin_start_index,timeout,locator,driver):
    asin_page = {}
    current_asin_index = 0
    for asin_idx in tqdm(range(asin_start_index,len(asin_list))):
        try:
            asin = asin_list[asin_idx]
            asin_page[asin] = getASINData(asin,timeout,locator,driver)
            current_asin_index = asin_idx
        except KeyboardInterrupt:
            print('user paused loop')
            break
        except:
            print(sys.exc_info())
            pass
    # print(asin_page)
    return asin_page,current_asin_index


In [14]:
dfASINList = pd.read_excel(r'Philips Top Products.xlsx',header = 0,engine = 'openpyxl')
asin_list = list(set(dfASINList['asin']))
asin_list

',
 'MIXDWGFRGAUFFFCE',
 'IRNFZV2ECJYZDMHH',
 'IRNFH2GTJHZN22Q8',
 'MIXFHEGTKEPSKWZE',
 'IRNFFQ7GDZ57GYMM',
 'SHVFJ9U3NV7TDAHZ',
 'PUTETPWWWJ4UPRG6',
 'IRNFZQHDHNZ7YGHN',
 'TMRFZY9RTJMURCDD',
 'SHVFGE36JUEHGNRQ',
 'IRNF3DQ66SRAPDWH',
 'FPREJYK6HAGFHGGY',
 'EPRFYKMHTCJTBNBA',
 'IRNFZV3ZMM594X7J',
 'BNLF7DQJTHWJPGXT',
 'EPRF5VE4GASGTQSE',
 'MIXEHC24GCZCJS6F',
 'ECKEME9XGSZBX9H4',
 'IRNFZQMD8C6GSBUX',
 'TMRFVSR8HHGZXGKT',
 'TMRFWCZKXAKTHZJA',
 'ICTFE23GXTHYWUNQ',
 'MIXFHEC3V53SHYCX',
 'AFREXBR9DP8MBXD9',
 'TMRFU7DPD6Y8GGEE',
 'FPRFKWQY2M2TSHDQ',
 'IRNFMECGZRZHGJNK',
 'PUTFGCNFJMJGP3VV',
 'BLBFY6NTFYZMGFWW',
 'TMRG23JJY4HQ2TUF',
 'ACCFM3YFG6FDUYMR',
 'MIXECAG4JGUH8WVS',
 'MJJFPFQKHVNGKYPY',
 'PWBFFESMH4SMCF9Z',
 'MIXE3GNWZWKGP3EZ',
 'TMRFVW7GNQGT6ZAZ',
 'HBLGF9AMQZSB3JVY',
 'PABFEK4GMCERKVCG',
 'ACCFVGXZB6XY9RYY',
 'TMRGYUTBVZS9BEXS',
 'HSTFG4SHRUMXAZUG',
 'SHVEKGZFTWXGBWUN',
 'PUTGFRE2GHBPT92K',
 'IRNFGH9WYAKHH8H3',
 'ICTGFUEY3HEPYFFY',
 'SHVFFU9ZQZC8WHEG',
 'TMRFW2Z3WGXEFTRK',
 'IRNFVZJ6

In [17]:
asin_list = asin_list[0:5]

In [20]:
locator = 'div>div>div>h1'
timeout = 10
asin_start_index = 0 #change to asin_current_index to resume loop from the point where it was paused
# asin_start_index = asin_current_index #
asin_details = {} ## comment this line and update asin_details with asin_page before restarting loop
asin_page,asin_current_index = getData(asin_list,asin_start_index,timeout,locator,driver)
asin_details.update(asin_page)
print("asin details", asin_page)

  0%|          | 0/5 [00:00<?, ?it/s]

asin details {'IRNFFUG4KY9P4FW5': {'product title': 'PHILIPS SNI07 1000 W Dry Iron  (Black)', 'product details': {'Sales Package': 1, 'Brand': 'PHILIPS', 'Model': 'SNI07', 'Type': 'Dry', 'Color': 'Black', 'Soleplate type': 'Black American Heritage', 'Spray': 'No', 'Steam Burst': 'No', 'Power Consumption': '1000 W', 'Height': '16 cm', 'Width': '29 cm', 'Depth': '13 cm', 'Description_product': 'Classic and Durable Design', 'timestamp': '2021-06-09 13:14:58.187664'}, 'feature details': {'timestamp': '2021-06-09 13:14:58.218915'}, 'hl_emi details': {'Highlights': 'Dry Iron\nBlack American Heritage Plate\nConsumes 1000 W', 'Easy Payment Options': 'Cash on Delivery\nNet banking & Credit/ Debit/ ATM card\nView Details', 'timestamp': '2021-06-09 13:14:58.315837'}, 'images': ['https://rukminim1.flixcart.com/image/416/416/juu4jgw0/iron/f/w/5/philips-sni07-original-imaffuftqhsayakd.jpeg?q=70', 'https://rukminim1.flixcart.com/image/416/416/juu4jgw0/iron/f/w/5/philips-sni07-original-imaffuftb6bbq2k

In [21]:
df_FlipkartData = pd.DataFrame.from_dict(asin_page,orient = 'index')

In [23]:
df_FlipkartData.reset_index(level = 0,inplace=True)

In [25]:
df_FlipkartData.rename(columns = {'index':'channel_sku_id'},inplace=True)

In [26]:
df_FlipkartData

,channel_sku_id,product title,product details,feature details,hl_emi details,images,num vids,timestamp
0,IRNFFUG4KY9P4FW5,PHILIPS SNI07 1000 W Dry Iron (Black),"{'Sales Package': 1, 'Brand': 'PHILIPS', 'Mode...",{'timestamp': '2021-06-09 13:14:58.218915'},{'Highlights': 'Dry Iron Black American Herita...,[https://rukminim1.flixcart.com/image/416/416/...,0,2021-06-09 13:14:58.315837
1,BLBFXZSXKDUJRRYT,"PHILIPS 12 W T-Bulb B22 LED Bulb (White, Pack...","{'Model Number': '12w T-Bulb Pk02', 'Model Nam...",{'timestamp': '2021-06-09 13:15:01.236517'},"{'Pack of': '1 2 3 4 5 6', 'Power Consumption'...",[https://rukminim1.flixcart.com/image/416/416/...,0,2021-06-09 13:15:01.355456
2,IRNFHD2FZYNN46KF,PHILIPS GC157 1100W 1100 W Dry Iron (Blue),"{'Sales Package': 'iron', 'Brand': 'PHILIPS', ...",{'timestamp': '2021-06-09 13:15:03.036210'},{'Highlights': 'Dry Iron taflone Plate Consume...,[https://rukminim1.flixcart.com/image/416/416/...,0,2021-06-09 13:15:03.117315
3,IRNFFTM4GNKH8D5Y,PHILIPS GC090 750 W Dry Iron (Blue),"{'Sales Package': 'User Manual, Warranty Card,...",{'timestamp': '2021-06-09 13:15:05.154286'},{'Highlights': 'Dry Iron Linished Soleplate Pl...,[https://rukminim1.flixcart.com/image/416/416/...,0,2021-06-09 13:15:05.233056
4,TMRFTZKKTTPEWYGY,PHILIPS AZB01953VNM6 Runtime: 45 min Trimmer f...,"{'Trimmer': nan, 'Brand': 'PHILIPS', 'Model Nu...",{'timestamp': '2021-06-09 13:15:06.564348'},{'Highlights': 'Blade Material: Stainless Stee...,[],0,2021-06-09 13:15:06.611893


In [124]:
df_FlipkartData.to_csv('philips_data.csv')

In [28]:
df_FlipkartData['timestamp'][0]

'2021-06-09 13:14:58.315837'

In [123]:
df_FlipkartData['product details'][14]

{'Manual, Concentrator, Main Unit': nan,
 'Brand': 'PHILIPS',
 'Model Number': 'HP8100/60',
 'Color': 'Blue',
 'Number of Speed Settings': '2',
 'Number of Heat Settings': '2',
 'Number of Attachments': '4',
 'Attachment Types': 'Concentrator',
 'Nozzle/Concentrator': 'yes',
 'Cold Air Feature': 'No',
 'Wattage': '1000 W',
 'Power Required (Volts)': '220-240',
 'Universal Voltage': 'Yes',
 'Removable Filter': 'Yes',
 'Hanging Loop': 'Yes',
 'Cord Length': '1.5 m',
 'Foldable': 'No',
 'Warranty Summary': '2-year guarantee',
 'Service Type': '2-year guarantee',
 'Covered in Warranty': '2-year guarantee',
 'Not Covered in Warranty': '2-year guarantee',
 'Description_product': 'Compact and ergonomic, this hairdryer benefits from a clever modern design. This results in a dryer that is light and easy to handle yet small enough to store virtually anywhere.This 1000W hairdryer creates the optimum level of airflow and gentle drying power, for beautiful results every day.The rubberised hook is l

In [ ]:
asin_details.update(asin_page)

In [ ]:
asin_details

In [ ]:
img_list,num_vids = getCatalogContent()
print("Images",img_list,"Number of videos",num_vids)